<a href="https://colab.research.google.com/github/noriakihanya/musicgen/blob/main/AudioCraft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Adapted from https://github.com/camenduru/MusicGen-colab
%cd /content
!git clone https://github.com/facebookresearch/audiocraft
%cd /content/audiocraft
!pip install -r requirements.txt

In [ ]:
# Click on the gradio link that appear.
%cd /content/audiocraft
!python app.py --share

# See also https://colab.research.google.com/drive/1fxGqfg96RBUvGxZ1XXN07s3DthrKUl4-?usp=sharing
# for a Colab demo using the underlying API instead.

In [1]:
!pip install -U audiocraft  # stable release
!pip install -U git+https://git@github.com/facebookresearch/audiocraft#egg=audiocraft  # bleeding edge
!pip install -e .  # or if you cloned the repo locally (mandatory if you want to train).
!git clone https://github.com/facebookresearch/audiocraft
!pip install pydub
!pip install mutagen

import gdown
gdown.download('https://drive.google.com/uc?id=1wVauTFZFFCqbiFzRAK2Th_Kn0p5HzOiG', 'MuseScore_General.sf2', quiet=False)
import shutil
!git clone https://github.com/noriakihanya/musicgen.git
#shutil.copy("/content/musicgen/musicgen.py","/usr/local/lib/python3.10/dist-packages/audiocraft/models")

import os
os.makedirs("/content/00_tmp/", exist_ok=True)
os.makedirs("/content/01_input", exist_ok=True)
os.makedirs("/content/02_output_midi", exist_ok=True)
os.makedirs("/content/03_output_mp3", exist_ok=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.9/423.9 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 97.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 35

  Cloning https://****@github.com/facebookresearch/audiocraft to /tmp/pip-install-fodkkwmk/audiocraft_1536f08c3dd045aa80114a3f1f69b7c4
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/facebookresearch/audiocraft' /tmp/pip-install-fodkkwmk/audiocraft_1536f08c3dd045aa80114a3f1f69b7c4
  Resolved https://****@github.com/facebookresearch/audiocraft to commit c5157b5bf14bf83449c17ea1eeb66c19fb4bc7f0
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 729.2/729.2 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 97.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for audiocraft: filename=audiocraft-1.0.0-py3-none-any.whl size=270765 sha256=4a24cc919ecc4dc74825f9d814c6fc279d671ff2044484810f4926807bf9797d
  Stored in directory: /tmp/pip-ephem-wheel-cache-a80s6d16/wheels/61/bb/15/cf53514254501b4472fb64d137bd3ab88737daf6917dfcbdc9
  Created wheel for encodec: filename=encodec

Downloading...
From: https://drive.google.com/uc?id=1wVauTFZFFCqbiFzRAK2Th_Kn0p5HzOiG
To: /content/MuseScore_General.sf2
100%|██████████| 216M/216M [00:02<00:00, 79.6MB/s]


Cloning into 'musicgen'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 45 (delta 19), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (45/45), 5.98 MiB | 8.74 MiB/s, done.
Resolving deltas: 100% (19/19), done.


In [2]:
#@title MusicGen API
import os
import shutil
import glob
import math
import librosa
import torch
import torchaudio
from audiocraft.models import MusicGen
from audiocraft.data.audio import audio_write, audio_read
from audiocraft.data.audio_utils import convert_audio
from audiocraft.utils.notebook import display_audio
from audiocraft.models import MultiBandDiffusion
USE_DIFFUSION_DECODER = True
if USE_DIFFUSION_DECODER:
  mbd = MultiBandDiffusion.get_mbd_musicgen()

from mutagen.mp3 import MP3
from pydub import AudioSegment

#for file in glob.glob('/content/03_output_mp3/*.mp3'):
#    os.remove(file)

#for file in glob.glob('/content/01_input/*.mp3'):
#  shutil.copy(file,"/content/03_output_mp3/")

for file in glob.glob('/content/01_input/*'):
    print(file)
    if ".wav"  in file:
      # 読み込み
      sound = AudioSegment.from_wav(file)
      # 変換
      file = "/content/01_input/musicgen.mp3"
      sound.export(file, format="mp3")

audio_time = MP3(file)
#duration = audio_time.info.length
duration = 10

model = MusicGen.get_pretrained('facebook/musicgen-melody')
model.set_generation_params(duration=duration)

melody_waveform, sr = torchaudio.load(file)
description = "'90s rock song with loud guitars and heavy drums'," #@param {type:"string"}
descriptions = [description]

melody_waveform = melody_waveform.unsqueeze(0).repeat(len(descriptions), 1, 1)
output = model.generate_with_chroma(
    descriptions,
    melody_wavs=melody_waveform,
    melody_sample_rate=sr,
    progress=True, return_tokens=True
)
display_audio(output[0], sample_rate=32000)
if USE_DIFFUSION_DECODER:
    out_diffusion = mbd.tokens_to_wav(output[1])
    display_audio(out_diffusion, sample_rate=32000)


for one_wav in out_diffusion:
    # Will save under {idx}.wav, with loudness normalization at -14 db LUFS.
    audio_write('/content/03_output_mp3/acp_musicgen', one_wav.detach().cpu().float(), model.sample_rate,
                format="mp3", strategy="loudness")

Downloading: "https://dl.fbaipublicfiles.com/encodec/Diffusion/mbd_musicgen_32khz.th" to /root/.cache/torch/hub/checkpoints/mbd_musicgen_32khz.th
100%|██████████| 1.53G/1.53G [00:10<00:00, 159MB/s]


/content/01_input/singingai_world.wav


Downloading: "https://dl.fbaipublicfiles.com/demucs/hybrid_transformer/955717e8-8726e21a.th" to /root/.cache/torch/hub/checkpoints/955717e8-8726e21a.th
100%|██████████| 80.2M/80.2M [00:01<00:00, 70.1MB/s]


CLIPPING /content/03_output_mp3/acp_musicgen happening with proba (a bit of clipping is okay): 0.00022499999613501132 maximum scale:  1.2986719608306885


In [ ]:
break
#-----------------------------------------------------------------------------------------------------------------
model = MusicGen.get_pretrained('melody')
model.set_generation_params(duration=duration)  # generate 8 seconds.
target_sr = 32000
target_ac = 1
processed_melodies = []
description = "An 80s driving pop song with heavy drums and synth pads in the background" #@param {type:"string"}
descriptions = [description]

melody, sr = audio_read(file)

if melody.dim() == 1:
  melody = melody[None]
melody = melody[..., :int(sr * duration)]
melody = convert_audio(melody, sr, target_sr, target_ac)
processed_melodies.append(melody)

#wav = model.generate_with_chroma(descriptions, melody[None].expand(len(descriptions), -1, -1), target_sr, progress=True)
wav = model.generate_with_chroma(descriptions, processed_melodies, target_sr, progress=True)

for idx, one_wav in enumerate(wav):
    # Will save under {idx}.wav, with loudness normalization at -14 db LUFS.
    audio_write('/content/03_output_mp3/acp_musicgen', one_wav.detach().cpu().float(), model.sample_rate,
                format="mp3", strategy="loudness")

audio = AudioSegment.from_file('/content/03_output_mp3/acp_musicgen.mp3')
# 音量(%)
volume_percent = 350

# 音量を調整して再生
mtf = audio + (20 * math.log10(volume_percent/100))
mtf.export('/content/03_output_mp3/acp_musicgen.mp3', format='mp3')

import IPython.display
IPython.display.Audio('/content/03_output_mp3/acp_musicgen.mp3')

In [ ]:
import math
audio = AudioSegment.from_file('/content/03_output_mp3/acp_musicgen.mp3')
# 音量(%)
volume_percent = 350

# 音量を調整して再生
mtf = audio + (20 * math.log10(volume_percent/100))
mtf.export('/content/03_output_mp3/acp_musicgen.mp3', format='mp3')

import IPython.display
IPython.display.Audio('/content/03_output_mp3/acp_musicgen.mp3')

In [ ]:
#@title インストール
# Best to make sure you have torch installed first, in particular before installing xformers.
# Don't run this if you already have PyTorch installed.
!pip install 'torch>=2.0'
# Then proceed to one of the following
!pip install -U audiocraft  # stable release
!pip install -U git+https://git@github.com/facebookresearch/audiocraft#egg=audiocraft  # bleeding edge
!pip install -e .  # or if you cloned the repo locally

!git clone https://github.com/facebookresearch/audiocraft

In [ ]:
#@title MusicGen API
import torchaudio
from audiocraft.models import MusicGen
from audiocraft.data.audio import audio_write

model = MusicGen.get_pretrained('melody')
model.set_generation_params(duration=30)  # generate 8 seconds.

#wav = model.generate_unconditional(4)    # generates 4 unconditional audio samples

#descriptions = ['happy rock', 'energetic EDM', 'sad jazz']
#wav = model.generate(descriptions)  # generates 3 samples.

melody, sr = torchaudio.load('/content/audiocraft/assets/bach.mp3')
#descriptions = ['happy rock', 'energetic EDM', 'sad jazz']
descriptions = ['acid jazz']
# generates using the melody from the given audio and the provided descriptions.
wav = model.generate_with_chroma(descriptions, melody[None].expand(len(descriptions), -1, -1), sr)

for idx, one_wav in enumerate(wav):
    # Will save under {idx}.wav, with loudness normalization at -14 db LUFS.
    audio_write(f'{idx}', one_wav.cpu(), model.sample_rate, strategy="loudness", loudness_compressor=True)

CLIPPING 0 happening with proba (a bit of clipping is okay): 0.0007479166379198432 maximum scale:  1.204222559928894


In [ ]:
#@title UI付き
# Adapted from https://github.com/camenduru/MusicGen-colab
%cd /content
!git clone https://github.com/facebookresearch/audiocraft
%cd /content/audiocraft
!pip install -r requirements.txt
# Click on the gradio link that appear.
!python app.py --share
# See also https://colab.research.google.com/drive/1fxGqfg96RBUvGxZ1XXN07s3DthrKUl4-?usp=sharing
# for a Colab demo using the underlying API instead.

In [ ]:
from audiocraft.models import MusicGen

# Using small model, better results would be obtained with `medium` or `large`.
model = MusicGen.get_pretrained('small')

In [ ]:
model.set_generation_params(
    use_sampling=True,
    top_k=250,
    duration=5
)

In [ ]:
#@title Unconditional Generation
from audiocraft.utils.notebook import display_audio

output = model.generate_unconditional(num_samples=2, progress=True)
display_audio(output, sample_rate=32000)

In [ ]:
#@title Music Continuation
import math
import torchaudio
import torch
from audiocraft.utils.notebook import display_audio

def get_bip_bip(bip_duration=0.125, frequency=440,
                duration=0.5, sample_rate=32000, device="cuda"):
    """Generates a series of bip bip at the given frequency."""
    t = torch.arange(
        int(duration * sample_rate), device="cuda", dtype=torch.float) / sample_rate
    wav = torch.cos(2 * math.pi * 440 * t)[None]
    tp = (t % (2 * bip_duration)) / (2 * bip_duration)
    envelope = (tp >= 0.5).float()
    return wav * envelope

In [ ]:
# Here we use a synthetic signal to prompt both the tonality and the BPM
# of the generated audio.
res = model.generate_continuation(
    get_bip_bip(0.125).expand(2, -1, -1),
    32000, ['Jazz jazz and only jazz',
            'Heartful EDM with beautiful synths and chords'],
    progress=True)
display_audio(res, 32000)

In [ ]:
# You can also use any audio from a file. Make sure to trim the file if it is too long!
prompt_waveform, prompt_sr = torchaudio.load("./assets/bach.mp3")
prompt_duration = 2
prompt_waveform = prompt_waveform[..., :int(prompt_duration * prompt_sr)]
output = model.generate_continuation(prompt_waveform, prompt_sample_rate=prompt_sr, progress=True)
display_audio(output, sample_rate=32000)

In [ ]:
#@title Text-conditional Generation
from audiocraft.utils.notebook import display_audio

output = model.generate(
    descriptions=[
        '80s pop track with bassy drums and synth',
        '90s rock song with loud guitars and heavy drums',
    ],
    progress=True
)
display_audio(output, sample_rate=32000)

In [ ]:
#@title Melody-conditional Generation
import torchaudio
from audiocraft.utils.notebook import display_audio

model = MusicGen.get_pretrained('melody')
model.set_generation_params(duration=8)

melody_waveform, sr = torchaudio.load("assets/bach.mp3")
melody_waveform = melody_waveform.unsqueeze(0).repeat(2, 1, 1)
output = model.generate_with_chroma(
    descriptions=[
        '80s pop track with bassy drums and synth',
        '90s rock song with loud guitars and heavy drums',
    ],
    melody_wavs=melody_waveform,
    melody_sample_rate=sr,
    progress=True
)
display_audio(output, sample_rate=32000)